## Import The Data

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv("laptop_price.csv")

## Understand The Data

In [ ]:
print(df.shape[0])
print(df.shape[1])

In [ ]:
df.isnull().sum()

In [ ]:
df.dtypes

In [ ]:
df.describe()

In [ ]:
for column in df.columns:
    print(f"{column} : {len(df[column].unique())}")

## Clean And Prepare The Data

In [ ]:
df.rename(columns = {'Price_euros' : 'Price'}, inplace= True)